In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

import numpy as np
import pandas as pd
from gekko import GEKKO
from Merge import merge
from copy import deepcopy
from DPModel import action_rollout

In [2]:
def to_list(arr):
    return [elem[0] for elem in arr]

In [3]:
min_bat_state = 0.0
max_bat_state = 13.0
min_charge = -7.0
max_charge = 7.0
deg_rate = 0.0
sbr_val = 0.10
fee = 1
verbose = True

start_time='2022-06-19 00:00:00'
end_time = '2022-06-19 23:00:00'
n = len(pd.date_range(start_time, end_time,freq='H'))

In [4]:
dfA = merge('h16')
dfB = merge('h28')
dfC = merge('k28')

yieldd_A = dfA['yield'].loc[start_time:end_time].to_numpy()
yieldd_B = dfB['yield'].loc[start_time:end_time].to_numpy()
yieldd_C = dfC['yield'].loc[start_time:end_time].to_numpy()

ini_bat_state_A = 0.0
ini_bat_state_B = 10.0
ini_bat_state_C = 0.0

price = dfA['SpotPriceDKK'].loc[start_time:end_time].to_numpy()/1000

optimals = [78.8,31.9,95.8] # SHO MPC wrt initial battery value

m = GEKKO()

In [5]:
# battery states
bat_state_A = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)
bat_state_B = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)
bat_state_C = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)

# cost states
cost_state_A = m.Array(m.Var, n)
cost_state_B = m.Array(m.Var, n)
cost_state_C = m.Array(m.Var, n)

# charge action array
charge_A = m.Array(m.Var, n, lb=min_charge, ub=max_charge)
charge_B = m.Array(m.Var, n, lb=min_charge, ub=max_charge)
charge_C = m.Array(m.Var, n, lb=min_charge, ub=max_charge)

# yield array
y_A = m.Array(m.Var, n)
y_B = m.Array(m.Var, n)
y_C = m.Array(m.Var, n)

# surplus array
s_A = m.Array(m.Var, n)
s_B = m.Array(m.Var, n)
s_C = m.Array(m.Var, n)
s_t = m.Array(m.Var, n)

# market price array
p = m.Array(m.Var, n)
# grid cost
grc = m.Array(m.Var, n)

In [6]:
# define initial battery state
m.Equation(bat_state_A[0] == ini_bat_state_A)
m.Equation(bat_state_B[0] == ini_bat_state_B)
m.Equation(bat_state_C[0] == ini_bat_state_C)

# define battery dynamics
m.Equation([bat_state_A[i+1] == bat_state_A[i]*(1-deg_rate)+charge_A[i] for i in range(n)])
m.Equation([bat_state_B[i+1] == bat_state_B[i]*(1-deg_rate)+charge_B[i] for i in range(n)])
m.Equation([bat_state_C[i+1] == bat_state_C[i]*(1-deg_rate)+charge_C[i] for i in range(n)])

# define yield
m.Equations([y_A[i] == yieldd_A[i] for i in range(n)])
m.Equations([y_B[i] == yieldd_B[i] for i in range(n)])
m.Equations([y_C[i] == yieldd_C[i] for i in range(n)])

# define surplus
m.Equations([s_A[i] == y_A[i] - charge_A[i] for i in range(n)])
m.Equations([s_B[i] == y_B[i] - charge_B[i] for i in range(n)])
m.Equations([s_C[i] == y_C[i] - charge_C[i] for i in range(n)])

# define market price
m.Equations([p[i] == price[i] for i in range(n)])

# define grid interaction
m.Equations([s_t[i] == s_A[i]+s_B[i]+s_C[i] for i in range(n)])
m.Equations([grc[i] == m.if3(s_t[i], -s_t[i]*(p[i]+fee), -s_t[i]*sbr_val*p[i]) for i in range(n)])

grid_cost = sum([grc[i] for i in range(n)])

# define cost states
#m.Equation([cost_state_A[i] == m.if3(s_t[i],-1*s_A[i]*p[i],0) for i in range(n)])
#m.Equation([cost_state_B[i] == m.if3(s_t[i],-1*s_B[i]*p[i],0) for i in range(n)])
#m.Equation([cost_state_C[i] == m.if3(s_t[i],-1*s_C[i]*p[i],0) for i in range(n)])

#m.Equation([cost_state_A[i] == m.if3(s_t[i],-1*s_A[i]*p[i],-s_A[i]*sbr_val*p[i]) for i in range(n)])
#m.Equation([cost_state_B[i] == m.if3(s_t[i],-1*s_B[i]*p[i],-s_B[i]*sbr_val*p[i]) for i in range(n)])
#m.Equation([cost_state_C[i] == m.if3(s_t[i],-1*s_C[i]*p[i],-s_C[i]*sbr_val*p[i]) for i in range(n)])

m.Equation([cost_state_A[i] == m.if3(-s_t[i],-s_A[i]*sbr_val*p[i],-1*s_A[i]*p[i]) for i in range(n)])
m.Equation([cost_state_B[i] == m.if3(-s_t[i],-s_B[i]*sbr_val*p[i],-1*s_B[i]*p[i]) for i in range(n)])
m.Equation([cost_state_C[i] == m.if3(-s_t[i],-s_C[i]*sbr_val*p[i],-1*s_C[i]*p[i]) for i in range(n)])

# define pareto optimality
m.Equation(sum(cost_state_A)<=optimals[0]-20.0)
m.Equation(sum(cost_state_B)<=optimals[1]-20.0)
m.Equation(sum(cost_state_C)<=optimals[2]-20.0)

# define cummulative costs
cumm_cost_house = sum([cost_state_A[i]+cost_state_B[i]+cost_state_C[i] for i in range(n)])

# define objective
m.Obj(grid_cost)

# Solver details
m.options.MAX_ITER = 10000
m.options.IMODE = 3
solvers = [1,3,2] 
for solver in solvers:
    try:
        if verbose:
            print(f'Trying solver {solver}')
        m.options.SOLVER = solver
        m.solve(disp=verbose)
        break 
    except Exception as e:
        if verbose:
            print(f"Solver {solver} failed with error: {e}")
else:
    print("All solvers failed")
    #m.open_folder()

Trying solver 1
apm 80.208.67.47_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          822
   Intermediates:            0
   Connections  :            0
   Equations    :          655
   Residuals    :          655
 
 Number of state variables:            822
 Number of total equations: -          654
 Number of slack variables: -          195
 ---------------------------------------
 Degrees of freedom       :            -27
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      2.63 NLPi:   63 Dpth:    0 Lvs:    3 Obj:  3.94E+02 Gap:       NaN
--

In [7]:
of = pd.DataFrame()
#of['charge_A'] = to_list(charge_A)
#of['bat_state_A'] = to_list(bat_state_A[:len(bat_state_A)-1])
#of['charge_B'] = to_list(charge_B)
#of['bat_state_B'] = to_list(bat_state_B[:len(bat_state_B)-1])
#of['charge_C'] = to_list(charge_C)
#of['bat_state_C'] = to_list(bat_state_C[:len(bat_state_C)-1])

of['s_A'] = to_list(s_A)
of['s_B'] = to_list(s_B)
of['s_C'] = to_list(s_C)
of['s_t'] = to_list(s_t)

of['price'] = to_list(p)

of['cost_state_A'] = to_list(cost_state_A)
of['cost_state_B'] = to_list(cost_state_B)
of['cost_state_C'] = to_list(cost_state_C)


of['cumm_cost_A'] = of['cost_state_A'].cumsum()
of['cumm_cost_B'] = of['cost_state_B'].cumsum()
of['cumm_cost_C'] = of['cost_state_C'].cumsum()

of['cost_houses'] = of['cost_state_A']+of['cost_state_B']+of['cost_state_C']
of['cumm_house_cost'] = of['cost_houses'].cumsum()

of['total_cost'] = to_list(grc)
of['total_cumm_cost'] = of['total_cost'].cumsum()

of.round(4)


,s_A,s_B,s_C,s_t,price,cost_state_A,cost_state_B,cost_state_C,cumm_cost_A,cumm_cost_B,cumm_cost_C,cost_houses,cumm_house_cost,total_cost,total_cumm_cost
0,-6.2000,2.2000,-7.3000,-11.3000,1.5404,9.5504,-3.3889,11.2448,9.5504,-3.3889,11.2448,17.4064,17.4064,28.7064,28.7064
1,-12.3000,-10.3000,-12.4000,-35.0000,1.4134,17.3849,14.5581,17.5263,26.9354,11.1693,28.7711,49.4694,66.8758,84.4694,113.1758
2,2.8000,3.7000,-6.5000,-0.0000,1.2313,-3.4476,-4.5558,0.8003,23.4877,6.6135,29.5715,-7.2031,59.6727,0.0000,113.1758
3,-3.1000,0.2000,2.9000,-0.0000,1.0400,0.3224,-0.2080,-3.0159,23.8101,6.4055,26.5556,-2.9015,56.7711,0.0000,113.1758
4,-10.8000,-3.0000,-10.2134,-24.0134,0.8971,9.6891,2.6914,9.1628,33.4992,9.0969,35.7184,21.5434,78.3145,45.5568,158.7325
5,3.6000,-6.2134,2.6134,-0.0000,0.8254,-2.9715,0.5129,-2.1572,30.5277,9.6098,33.5612,-4.6159,73.6987,0.0000,158.7325
6,-6.1781,0.9134,-3.2000,-8.4647,0.8373,5.1727,-0.7648,2.6792,35.7003,8.8450,36.2405,7.0871,80.7858,15.5518,174.2843
7,-10.1000,-9.4000,-11.3000,-30.8000,0.7333,7.4059,6.8926,8.2858,43.1063,15.7376,44.5263,22.5844,103.3702,53.3844,227.6688
8,3.7000,-7.9000,4.2000,-0.0000,0.6735,-2.4918,0.5320,-2.8285,40.6145,16.2697,41.6978,-4.7882,98.5820,0.0000,227.6688
9,-4.8477,6.3000,-1.4523,-0.0000,0.7013,0.3400,-4.4185,0.1019,40.9545,11.8512,41.7997,-3.9767,94.6053,0.0000,227.6688


# Defining costs using OOP for all buyers, all sellers

In [572]:
class Buyer:
    def __init__(self, name, price, grid_price, demand):
        self.name = name
        self.price = price
        self.grid_price = grid_price
        self.demand = demand
        
        self.total_purchase = 0
        self.peer_purchase = 0
        self.grid_purchase = 0
    
    def purchase(self, quantity):
        self.peer_purchase += quantity
        self.total_purchase += quantity
    
    def unfulfilled_demand(self):
        return self.demand - self.total_purchase
    
    def purchase_from_grid(self):
        demand_gap = self.unfulfilled_demand()
        cost = demand_gap * self.grid_price
        self.grid_purchase += demand_gap
        self.total_purchase += demand_gap
        return cost
    
    def purchase_from_peers(self):
        return self.peer_purchase*self.price
    
    def total_cost(self):
        return self.purchase_from_grid()+self.purchase_from_peers()

In [573]:
class Seller:
    def __init__(self, name, price, sbr, supply):
        self.name = name
        self.price = price
        self.sbr = sbr
        self.supply = supply
        
        self.total_sale = 0
        self.peer_sale = 0
        self.grid_sale = 0
    
    def sell(self, quantity):
        self.total_sale += quantity
        self.peer_sale += quantity
    
    def unsold_supply(self):
        return self.supply - self.total_sale
    
    def sell_to_grid(self):
        unsold = self.unsold_supply()
        revenue = unsold * self.price * self.sbr
        self.total_sale += unsold
        return revenue
    
    def sell_to_peers(self):
        return self.peer_sale*self.price
    
    def total_cost(self):
        return self.sell_to_peers() + self.sell_to_grid()


In [574]:
def energy_exchange(buyers, sellers):
    if not buyers or not sellers:
        return buyers

    buyer = max(buyers, key=lambda b: b.demand) # Find the buyer with the highest demand
    
    if not sellers:
        return [buyer] + energy_exchange([b for b in buyers if b != buyer], [])
    
    seller = max(sellers, key=lambda s: s.supply) # Find the seller with the highest supply
    
    if buyer.demand > seller.supply:
        buyer.purchase(seller.supply)
        seller.sell(seller.supply)
        sellers.remove(seller)
        return [buyer] + energy_exchange(buyers, sellers)
    
    elif seller.supply > buyer.demand:
        seller.sell(buyer.demand)
        buyer.purchase(buyer.demand)
        buyers.remove(buyer)
        return [seller] + energy_exchange(buyers, sellers)
    
    else:
        buyer.purchase(buyer.demand)
        seller.sell(seller.supply)
        buyers.remove(buyer)
        sellers.remove(seller)
        return [buyer, seller] + energy_exchange(buyers, sellers)


In [593]:
class EnergyMarket:
    def __init__(self, participants, price, grid_price):
        self.buyers = {}
        self.sellers = {}
        self.participants = participants
        self.price = price
        self.grid_price = grid_price
        
        for name, value in self.participants.items():
            if value < 0:
                buyer = Buyer(name, self.price, self.grid_price, -value)
                self.buyers[name] = buyer
            else:
                seller = Seller(name, self.price, 0.10, value)
                self.sellers[name] = seller
        
        self.buyers_list = list(self.buyers.values())
        self.sellers_list = list(self.sellers.values())
        
    def get_buyers(self):
        return self.buyers
    
    def get_sellers(self):
        return self.sellers

    def cal_costs(self):
        results = energy_exchange(self.buyers_list,self.sellers_list)
        return self.buyers, self.sellers
    
    def get_total_costs(self):
        buy_dict, sell_dict = self.cal_costs()
        costs = {}
        for name, buyer in buy_dict.items():
            costs[name] = buyer.total_cost()
        for name, seller in sell_dict.items():
            costs[name] = -seller.total_cost()
        return costs

In [594]:
nf = pd.DataFrame()
nf['A'] = to_list(s_A)
nf['B'] = to_list(s_B)
nf['C'] = to_list(s_C)

In [600]:
new = pd.DataFrame()
for i in range(len(nf)):
    new = new.append(EnergyMarket(nf.iloc[i].to_dict(),price[i],price[i]).get_total_costs(),ignore_index=True)

In [601]:
nf['OA'] = new['A'].to_list()
nf['OB'] = new['B'].to_list()
nf['OC'] = new['C'].to_list()
nf['cumm_OA'] = nf['OA'].cumsum()
nf['cumm_OB'] = nf['OB'].cumsum()
nf['cumm_OC'] = nf['OC'].cumsum()

In [602]:
nf

,A,B,C,cA,OA,cB,OB,cC,OC,cumm_OA,cumm_OB,cumm_OC
0,-5.500000,2.200000,-7.300000,13.972145,8.472145,-3.388858,-0.338886,16.344847,11.244847,8.472145,-0.338886,11.244847
1,-9.221547,-6.725251,-9.053202,22.255374,13.033827,16.230789,9.505537,21.849089,12.795887,21.505972,9.166652,24.040734
2,-0.278453,3.125251,-2.846798,0.342859,0.342859,-3.848122,-0.384812,3.505262,3.505262,21.848831,8.781839,27.545996
3,-3.800000,-2.800000,-4.100000,7.751886,3.951886,5.711916,2.911916,8.363877,4.263877,25.800717,11.693755,31.809873
4,-7.914474,-3.049768,-7.335758,15.014865,7.100391,5.785836,2.736069,13.916961,6.581202,32.901108,14.429824,38.391075
5,-10.400000,-9.700000,-10.500000,18.984472,8.584472,17.706671,8.006671,19.167015,8.667015,41.485580,22.436495,47.058090
6,-3.173759,-0.394525,3.568284,2.657261,2.657261,0.330320,0.330320,-2.987582,-0.298758,44.142842,22.766815,46.759332
7,3.552789,2.440317,-5.993106,-2.605118,-0.260512,-1.789387,-0.178939,4.394505,4.394505,43.882330,22.587877,51.153837
8,0.767726,-3.128018,2.360292,-0.517025,-0.051703,2.106564,2.106564,-1.589539,-0.158954,43.830627,24.694440,50.994884
9,-10.200000,-7.700000,-8.400000,17.353770,7.153770,13.100395,5.400395,14.291340,5.891340,50.984397,30.094835,56.886223


### Theory: Defining costs using logic/expert system

In [168]:
def round_one_decimal(number):
    """
    Returns the float that ignores everything beyond the first decimal

    Return type: float
    """
    return int(number*10)/10

In [169]:
def doub_seller(s1,s2,b,pr,grpr,sbr): # s1 and s2 want to sell, b is the solo buyer
    
    if abs(s1) >= abs(s2):            # s1 selling more than s2
        left_over = abs(s1)-abs(b)         # s1 tries to fit to the demand of b
        
        if round_one_decimal(left_over) >=0: # s1 fits the demand of b
            
            c_b = -b*pr
            c_s1 = b*pr + left_over*-pr*sbr
            c_s2 = -s2*pr*sbr
        else:                                # s2 joins s1 to fit demand of b
            
            c_s1 = -s1*pr
            
            left_over_2 = s2 - abs(left_over)
            
            if round_one_decimal(left_over_2) >= 0: # s1 and s2 matched demand of b
                c_b = -b*pr
                c_s2 = left_over_2*-pr*sbr + abs(left_over)*-pr
            
            else:                       # s2 wasn't enough
                c_b = (s1+s2)*pr + -left_over_2*grpr
                c_s2 = s2*-pr
        
        return c_s1,c_s2,c_b 
    
    else:
        c_s2,c_s1,c_b = doub_seller(s2,s1,b,pr,grpr,sbr)
        return c_s1,c_s2,c_b 

In [170]:
def solo_seller(s,b1,b2,pr,grpr,sbr): # a is the solo seller, # b is greater than or equal to c

    if abs(b1)>=abs(b2):         # B's demand is greater than C's demand
        left_over = s-abs(b1)        # A sells all to B first
        
        if round_one_decimal(left_over) >= 0:           # B buys from A & A still has something left
            c_b1 = -b1*pr
            
            left_over_2 = left_over-abs(b2) 
            
            if round_one_decimal(left_over_2) >= 0.0:    # C also buys from A & A still has something left
                c_b2 = -b2*pr
                c_s = -left_over_2*pr*sbr+(abs(b1)+abs(b2))*-pr

            else:                   # C also buys from A & has to buy more from grid 
                c_b2 = left_over*pr+abs(left_over_2)*grpr
                c_s = s*pr

        else:                       # A was not able to meet B's demand
            c_s = -s*pr
            c_b2 = -b2*grpr
            c_b1 = abs(left_over)*grpr+s*pr
        
        return c_s,c_b1,c_b2
    
    else:
        c_s,c_b2,c_b1 = solo_seller(s,b2,b1,pr,grpr,sbr)
        return c_s,c_b1,c_b2

In [560]:
A = of['s_A'].to_list()
B = of['s_B'].to_list()
C = of['s_C'].to_list()
T = of['s_t'].to_list()
price = of['price'].to_list()
grid_price = [i+1 for i in price]
sbr = 0.1

In [561]:
cA = []
cB = []
cC = []
for i in range(len(A)):
    
    if A[i]<=0 and B[i]<=0 and C[i]<=0: # all of them need to buy
        cA.append(-A[i]*grid_price[i])
        cB.append(-B[i]*grid_price[i])
        cC.append(-C[i]*grid_price[i])
    
    elif A[i]>0 and B[i]>0 and C[i]>0: # all of them need to sell
        cA.append(-A[i]*price[i]*sbr)
        cB.append(-B[i]*price[i]*sbr)
        cC.append(-C[i]*price[i]*sbr)
    
    
    elif A[i]>0 and B[i]<=0 and C[i]<=0: # A is seller and B and C are buying
        c_s,c_b1,c_b2 = solo_seller(A[i],B[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_s)
        cB.append(c_b1)
        cC.append(c_b2)
    
    elif B[i]>0 and A[i]<=0 and C[i]<=0: # B is seller and A and C are buying
        c_s,c_b1,c_b2 = solo_seller(B[i],A[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_b1)
        cB.append(c_s)
        cC.append(c_b2)
    
    elif C[i]>0 and A[i]<=0 and B[i]<=0: # C is seller and A and B are buying
        c_s,c_b1,c_b2 = solo_seller(C[i],A[i],B[i],price[i],grid_price[i],sbr)
        cA.append(c_b1)
        cB.append(c_b2)
        cC.append(c_s)
    
    elif A[i]>0 and B[i]>0 and C[i]<=0:
        c_s1,c_s2,c_b = doub_seller(A[i],B[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_s1)
        cB.append(c_s2)
        cC.append(c_b)
    
    elif A[i]>0 and C[i]>0 and B[i]<=0:
        c_s1,c_s2,c_b = doub_seller(A[i],C[i],B[i],price[i],grid_price[i],sbr)
        cA.append(c_s1)
        cB.append(c_b)
        cC.append(c_s2)
    
    else: # B[i]>0 and C[i]>0 and A[i]<=0:
        c_s1,c_s2,c_b = doub_seller(B[i],C[i],A[i],price[i],grid_price[i],sbr)
        cA.append(c_b)
        cB.append(c_s1)
        cC.append(c_s2)

In [562]:
tf = pd.DataFrame()
tf['A'] = A
tf['B'] = B
tf['C'] = C
tf['S'] = to_list(s_t)
tf['pr'] = price
tf['gpr'] = grid_price
tf['cA'] = cA
tf['cB'] = cB
tf['cC'] = cC
tf['cumm_A'] = tf['cA'].cumsum()
tf['cumm_B'] = tf['cB'].cumsum()
tf['cumm_C'] = tf['cC'].cumsum()
tf.round(3)

,A,B,C,S,pr,gpr,cA,cB,cC,cumm_A,cumm_B,cumm_C
0,-5.500,2.200,-7.300,-10.6,1.540,2.540,13.972,-3.389,16.345,13.972,-3.389,16.345
1,-9.222,-6.725,-9.053,-25.0,1.413,2.413,22.255,16.231,21.849,36.228,12.842,38.194
2,-0.278,3.125,-2.847,0.0,1.231,2.231,0.343,-3.848,3.505,36.570,8.994,41.699
3,-3.800,-2.800,-4.100,-10.7,1.040,2.040,7.752,5.712,8.364,44.322,14.706,50.063
4,-7.914,-3.050,-7.336,-18.3,0.897,1.897,15.015,5.786,13.917,59.337,20.492,63.980
5,-10.400,-9.700,-10.500,-30.6,0.825,1.825,18.984,17.707,19.167,78.322,38.198,83.147
6,-3.174,-0.395,3.568,0.0,0.837,1.837,2.657,0.330,-2.988,80.979,38.529,80.159
7,3.553,2.440,-5.993,0.0,0.733,1.733,-2.605,-1.789,4.395,78.374,36.739,84.554
8,0.768,-3.128,2.360,0.0,0.673,1.673,-0.517,2.107,-1.590,77.857,38.846,82.964
9,-10.200,-7.700,-8.400,-26.3,0.701,1.701,17.354,13.100,14.291,95.210,51.946,97.256
